# Reconstruction error

In [ ]:
include("stuff.jl")

In [ ]:
pca_recon_error_1_sp = loadCube(joinpath(cubes_base_dir, "pca_recon_error_1_sp"))
pca_recon_error_1_lat = loadCube(joinpath(cubes_base_dir, "pca_recon_error_1_lat"))
pca_recon_error_2_sp = loadCube(joinpath(cubes_base_dir, "pca_recon_error_2_sp"))
pca_recon_error_2_lat = loadCube(joinpath(cubes_base_dir, "pca_recon_error_2_lat"))
pca_recon_error_3_sp = loadCube(joinpath(cubes_base_dir, "pca_recon_error_3_sp"))
pca_recon_error_3_lat = loadCube(joinpath(cubes_base_dir, "pca_recon_error_3_lat"))

In [ ]:
R"""
library(viridis)
library(rworldmap)
data(coastsCoarse, package = "rworldmap")

options(device = function(filename=getOption('rcalljl_filename'),...) png(filename, width = 700, height = 400, ...))

elem_col = "black" # "gray50"
from_to = 120:700

#pal <- c("#fff7fb", "#ece7f2", "#d0d1e6", "#a6bddb", "#74a9cf", "#3690c0", "#0570b0", "#045a8d", "#023858")
#pal <- colorRampPalette(colors = pal)(100)
pal <- rev(viridis(100))

data1 <- $(pca_recon_error_1_sp[:, :] |> collect |> x -> x[:, end:-1:1] |> missing_to_nan .|> Float64)
data2 <- $(pca_recon_error_2_sp[:, :] |> collect |> x -> x[:, end:-1:1] |> missing_to_nan .|> Float64)
data3 <- $(pca_recon_error_3_sp[:, :] |> collect |> x -> x[:, end:-1:1] |> missing_to_nan .|> Float64)

ex <- range(c(range(data1, na.rm = TRUE), range(data2, na.rm = TRUE), range(data3, na.rm = TRUE)))
ex <- c(0, ex[2] * 0.3)

upper_margin = 2.1

data_lat1 <- $(pca_recon_error_1_lat[:] |> collect |> x -> x[:, end:-1:1] |> missing_to_nan .|> Float64)
data_lat2 <- $(pca_recon_error_2_lat[:] |> collect |> x -> x[:, end:-1:1] |> missing_to_nan .|> Float64)
data_lat3 <- $(pca_recon_error_3_lat[:] |> collect |> x -> x[:, end:-1:1] |> missing_to_nan .|> Float64)
data_lat1[is.na(data_lat1)] <- 0
data_lat1 <- rev(as.vector(data_lat1))
data_lat2[is.na(data_lat2)] <- 0
data_lat2 <- rev(as.vector(data_lat2))
data_lat3[is.na(data_lat3)] <- 0
data_lat3 <- rev(as.vector(data_lat3))

pdf($(joinpath(fig_path, "recon_error_pc123.pdf")), width = 7, height = 9)
layout(matrix(c(1:6), byrow = TRUE, ncol = 2), widths = c(1, 0.2))


plot_map <- function(data, ex, pal) {
    par(mar = c(4.2, 4.3, upper_margin, 0), xpd = FALSE)
    image(
        seq(from = -180, to = 180, length.out = dim(data)[1] + 1),
        seq(from = -90 * ((360 - 120) / 360),
            to   =  90 * (360 - 20) / 360,
            length.out = length(from_to) + 1),
        #seq(from = 120, to = 700, length.out = length(from_to) + 1),
        data[,from_to], 
        zlim = ex, 
        col = pal, 
        #xaxt = "n", 
        #yaxt = "n",
        xlab = expression("Lon " * group("[", degree, "]")),
        ylab = expression("Lat " * group("[", degree, "]")),
        las = 1,
        bty = "n", 
        useRaster = TRUE
    )
    image(
        seq(from = -180, to = 180, length.out = dim(data)[1] + 1),
        seq(from = -90 * ((360 - 120) / 360),
            to   =  90 * (360 - 20) / 360,
            length.out = length(from_to) + 1),
        data[,from_to], 
        zlim = c(ex[2], 99999), 
        col = pal[length(pal)], 
        xaxt = "n", 
        yaxt = "n",
        xlab = NULL,
        ylab = NULL,
        bty = "n", 
        useRaster = TRUE,
        add = TRUE
    )
    plot(coastsCoarse, add = TRUE)
}


#box(col = elem_col)
#box(col = elem_col)
plot_margin <- function(data_lat, ex, pal){
    # The right margin is required, because pdf rendering of raster objects and vector objects at the border is weird.
    par(mar = c(4.2, 0, upper_margin, 0.1))
    image(
        seq(from = 0, to = ex[2], length.out = 101),
        0:1,
        t(t(seq(from = ex[1], to = ex[2], length.out = 100))),
        col = pal, bty = "n", 
        xaxt = "n", yaxt = "n",
        ylab = NA,
        #xlim = c(0, max(data_lat[from_to]) * 0.9),
        #xlim = c(0, 1.2),
        xlab = expression("MSE " * group("[", "std" ^ 2, "]")),
        useRaster = TRUE
    )
    par(xpd = NA)
    polygon(
        # we need to extend the range, so that the background image is generously covered.
        x = c(0,   ex[2] + 1, ex[2] + 1,    0, data_lat[from_to]),
        y = c(1.01,   1.01,  -0.0, -0.0, seq(from = 0, to = 1, length.out = length(from_to))),
        col = "white",
        # we also need a border because rendering the polygon without border looks weird on some devices
        border = "white", 
        lwd = 1
    )
    polygon(
        x = c(data_lat[from_to]),
        y = seq(from = 0, to = 1, length.out = length(from_to)),
        lwd = 1
    )
    axis(1)
}

plot_map(data1, ex, pal)
plot_margin(data_lat1, ex, pal)
plot_map(data2, ex, pal)
plot_margin(data_lat2, ex, pal)
plot_map(data3, ex, pal)
plot_margin(data_lat3, ex, pal)


#grid.lines(1 / 1.2, c(2/3 + 0.07, 3/3 - 0.005))
#grid.lines(1 / 1.2, c(1/3 + 0.07, 2/3 - 0.005))
#grid.lines(1 / 1.2, c(0/3 + 0.07, 1/3 - 0.005))
grid.text("(a)",     0, 3/3, just = c(-3.0, 1.5))
grid.text("(b)", 1/1.2, 3/3, just = c(-0.1, 1.5))
grid.text("(c)",     0, 2/3, just = c(-3.0, 1.5))
grid.text("(d)", 1/1.2, 2/3, just = c(-0.1, 1.5))
grid.text("(e)",     0, 1/3, just = c(-3.0, 1.5))
grid.text("(f)", 1/1.2, 1/3, just = c(-0.1, 1.5))
#box(col = elem_col)
dev.off()
"""

show_pdf("http://127.0.0.1:8333/files/results/fig/recon_error_pc123.pdf", width = 700, height = 1000)